In [0]:
# Heavily inspired by: https://github.com/pytorch/examples/blob/master/mnist/main.py
import os
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [0]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.LeakyReLU(),
            nn.Linear(256, 10),
            nn.Sigmoid(),
        )
        
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        output = self.model(img_flat)
        return output

In [0]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_loader.dataset)
    print('Train Epoch: {}, Average Loss: {:.6f}'.format(
        epoch, train_loss))

In [0]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
def main():
    # Load data
    root = './data'
    if not os.path.exists(root):
        os.mkdir(root)

    # normalizes values to interval [-1, 1]
    trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
    test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

    batch_size = 64
    train_loader = torch.utils.data.DataLoader(
                     dataset=train_set,
                     batch_size=batch_size,
                     shuffle=True)
    test_loader = torch.utils.data.DataLoader(
                    dataset=test_set,
                    batch_size=batch_size,
                    shuffle=False)
    
    # Cuda stuff
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Device is " + str(device) + ".")
    
    # Training
    epochs = 10
    model = NN().to(device)
    criterion = torch.nn.CrossEntropyLoss(reduction='sum')
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, criterion, epoch)
        test(model, device, test_loader, criterion)
    
if __name__ == '__main__':
    main()

Device is cuda.
Train Epoch: 1, Average Loss: 1.568872
Test set: Average loss: 1.5123, Accuracy: 9403/10000 (94%)
Train Epoch: 2, Average Loss: 1.509146
Test set: Average loss: 1.5087, Accuracy: 9436/10000 (94%)
Train Epoch: 3, Average Loss: 1.498728
Test set: Average loss: 1.4958, Accuracy: 9604/10000 (96%)
Train Epoch: 4, Average Loss: 1.492732
Test set: Average loss: 1.4922, Accuracy: 9622/10000 (96%)
Train Epoch: 5, Average Loss: 1.489207
Test set: Average loss: 1.4888, Accuracy: 9659/10000 (97%)
Train Epoch: 6, Average Loss: 1.485221
Test set: Average loss: 1.4897, Accuracy: 9656/10000 (97%)
Train Epoch: 7, Average Loss: 1.484429
Test set: Average loss: 1.4960, Accuracy: 9566/10000 (96%)
Train Epoch: 8, Average Loss: 1.481736
Test set: Average loss: 1.4868, Accuracy: 9697/10000 (97%)
Train Epoch: 9, Average Loss: 1.481156
Test set: Average loss: 1.4853, Accuracy: 9698/10000 (97%)
Train Epoch: 10, Average Loss: 1.479651
Test set: Average loss: 1.4827, Accuracy: 9730/10000 (97%)
